In [2]:
import warnings
warnings.filterwarnings("ignore")
import copy
import pandas as pd
from tb_incubator.constants import set_project_base_path, QUANTILES, image_path, ImplementCDR
from tb_incubator.calibrate import (
    plot_output_ranges
)
from tb_incubator.scenario_utils import (
    get_quantile_outputs, 
    load_idata, 
    extract_and_save_idata, 
    run_model_for_scenario, calculate_waic_comparison
)
from tb_incubator.plotting import get_combined_plot, overlay_plots
from tb_incubator.input import load_targets

project_paths = set_project_base_path("../tb_incubator/")
calib_out = project_paths["OUT_PATH"]

In [3]:
targets = load_targets()
out_req = ["notification_log", "adults_prevalence_pulmonary_log"]
params = {
    "start_population_size": 1.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}

## WAIC

In [ ]:
file_suffix = "25000d10000t_01rs"
model_configs = {
    f'no_cdr_{file_suffix}': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.NONE,
    },
    f'cdr_notif_{file_suffix}': { 
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.ON_NOTIFICATION,
    },
    f'cdr_detect_{file_suffix}': {
        'apply_diagnostic_capacity': True,
        'xpert_improvement': True,
        'apply_cdr': ImplementCDR.ON_DETECTION,
    }
}

In [ ]:
inference_data_dict = load_idata(calib_out, model_configs)
extract_and_save_idata(inference_data_dict, calib_out)
xpert_outputs = run_model_for_scenario(params, calib_out, model_configs, QUANTILES)

In [ ]:
waic_results = calculate_waic_comparison(xpert_outputs)
waic_results
#waic_results.to_csv('results_table_waic.csv', index=True)

## Plots

In [ ]:
file_names = [f"no_cdr_{file_suffix}",
              f"cdr_notif_{file_suffix}",
              f"cdr_detect_{file_suffix}"
            ]

assumption_titles = {
    f"no_cdr_{file_suffix}": "No CDR",
    f"no_cdr_{file_suffix}": "No CDR",
    f"cdr_notif_{file_suffix}": "CDR on notification",
    f"cdr_notif_{file_suffix}": "CDR on notification",
    f"cdr_detect_{file_suffix}": "CDR on treatment commencement",
    f"cdr_detect_{file_suffix}": "CDR on treatment commencement"
}
titles = [assumption_titles[name] for name in file_names]



['No CDR', 'CDR on notification', 'CDR on treatment commencement']

In [54]:
quantile_outputs = get_quantile_outputs(file_names, calib_out)

plot_indicators = {
    'notification': ['notification'],
    'adults_prev': ['adults_prevalence_pulmonary'],
    'incidence': ['incidence'],
    'ltbi': ['percentage_latent'],
    'mortality': ['mortality'],
    'pdetect': ['passive_notification_raw']
}

plots = {}
for indicator_key, indicator_list in plot_indicators.items():
    plots[indicator_key] = {}
    for name in file_names:
        name_index = file_names.index(name)
        
        if name_index == 0:  
            color = "217,95,2"
        elif name_index == 1: 
            color = "27, 158, 119"  
        else:  
            color = "100, 150, 200" 
        
        plots[indicator_key][name] = plot_output_ranges(
            quantile_outputs[name], targets, indicator_list, 1, 2010, 2035, 2013,
            show_legend=False, show_target_data=True, show_title=False, colour=color
        )

## Compare all assumptions

In [68]:
scenarios = [
    ('no_cdr_25000d10000t_01rs', 'No CDR'),
    ('cdr_notif_25000d10000t_01rs', 'CDR on notification'),
    ('cdr_detect_25000d10000t_01rs', 'CDR on treatment commencement'),
]

notif_prev = [
    plots[plot_type][scenario_key]
    for scenario_key, _ in scenarios
    for plot_type in ['notification', 'adults_prev']
]

plot_titles = [title for _, title in scenarios for _ in range(2)]

comb_plot = get_combined_plot(
    plot_list=notif_prev,
    n_cols=2,
    subplot_titles=plot_titles,
    shared_yaxes=False,
    shared_xaxes=False,
    horizontal_spacing=0.15,
    vertical_spacing=0.10
)

In [ ]:
positions = [(1, 1), (2, 1), (3, 1), (1, 2), (2, 2), (3, 2)]
years_data = [
    (2017, "1st inventory study"),
    (2023, "2nd inventory study")
]

for row, col in positions:
    for year, text in years_data:
        comb_plot.add_vline(x=year, line_dash="dot", row=row, col=col, 
                       line_color="rgba(148, 145, 145, 0.5)", line_width=2,
                       annotation_text=text, annotation_textangle=-90,
                       annotation_position="bottom right",
                       annotation_font_size=8, annotation_font_color="#4E4B4B")

comb_plot

## Overlay comparison

In [71]:
targets = load_targets()
out_req = ["notification_log", "adults_prevalence_pulmonary_log"]
file_id = "25000d10000t_01rs"
file_suffixes = [f'no_extensions_{file_id}',
                 f'no_cdr_{file_id}',
                 f'cdr_notif_noxpert_{file_id}',
                 f'cdr_notif_{file_id}',
                 f'cdr_detect_noxpert_{file_id}',
                 f'cdr_detect_{file_id}']

quantile_outputs = get_quantile_outputs(file_suffixes, calib_out)

In [10]:
from tb_incubator.plotting import overlay_plots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

epi_plots = {}
indicators = ['notification', 'adults_prevalence_pulmonary', 'incidence', 'mortality']

for file in file_suffixes:
    epi_plots[file] = {}  
    for indicator in indicators:
        epi_plots[file][indicator] = plot_output_ranges(
            quantile_outputs[file], targets, 
            [f'{indicator}'], 1, 2010, 2035, 2013, show_legend=False, show_title=False, show_target_data=False
        )
        
notif_none = epi_plots[f'no_extensions_{file_id}']['notification']
notif_none_xpert = epi_plots[f'no_cdr_{file_id}']['notification']
notif_notif = epi_plots[f'cdr_notif_noxpert_{file_id}']['notification']
notif_notif_xpert = epi_plots[f'cdr_notif_{file_id}']['notification']
notif_detect = epi_plots[f'cdr_detect_noxpert_{file_id}']['notification']
notif_detect_xpert = epi_plots[f'cdr_detect_{file_id}']['notification']

fig1 = overlay_plots([notif_none, notif_none_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=True, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig2 = overlay_plots([notif_notif, notif_notif_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=False, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig3 = overlay_plots([notif_detect, notif_detect_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=False, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig1.add_trace(
    go.Scatter(
        x=targets['notif2000'].index,
        y=targets['notif2000'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Case notifications (WHO)",
        legendgroup="Target",
    )
)
fig2.add_trace(
    go.Scatter(
        x=targets['notif2000'].index,
        y=targets['notif2000'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Case notifications (WHO)",
        legendgroup="Target",
    )
)
fig3.add_trace(
    go.Scatter(
        x=targets['notif2000'].index,
        y=targets['notif2000'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Case notifications (WHO)",
        legendgroup="Target",
    )
)


#fig.write_image(image_path / f'xpert_compare.png', width=600, height=400,scale=3)

In [11]:
stacked_fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[f"<b>No CDR</b>", f"<b>CDR on notification</b>", f"<b>CDR on treatment commencement</b>"],  # Update with your actual titles
    vertical_spacing=0.08,  # Adjust spacing between subplots
    shared_xaxes=True  # Share x-axis if they have the same range
)

# Add traces from fig1 to row 1
for trace in fig1.data:
    stacked_fig.add_trace(trace, row=1, col=1)

# Add traces from fig2 to row 2
for trace in fig2.data:
    trace.showlegend = False  # Hide duplicate legends
    stacked_fig.add_trace(trace, row=2, col=1)

# Add traces from fig3 to row 3
for trace in fig3.data:
    trace.showlegend = False  # Hide duplicate legends
    stacked_fig.add_trace(trace, row=3, col=1)

# Update layout
stacked_fig.update_layout(
    height=900,  # Adjust total height
    showlegend=True,
    font_family="Space Grotesk",
    title_font_family="Space Grotesk",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

# Update y-axis titles if needed
stacked_fig.update_yaxes(title_text="No. of case notification", row=1, col=1)
stacked_fig.update_yaxes(title_text="No. of case notification", row=2, col=1)
stacked_fig.update_yaxes(title_text="No. of case notification", row=3, col=1)

stacked_fig.update_xaxes(title_text="Year", row=3, col=1)  # Only bottom x-axis gets label

stacked_fig.show()


positions = [(1, 1), (2, 1), (3, 1)]
years_data = [
    (2017, "1st inventory study"),
    (2023, "2nd inventory study")
]

for row, col in positions:
    for year, text in years_data:
        stacked_fig.add_vline(x=year, line_dash="dot", row=row, col=col, 
                       line_color="rgba(148, 145, 145, 0.5)", line_width=1,
                       annotation_text=text, annotation_textangle=-90,
                       annotation_position="bottom right",
                       annotation_font_size=9, annotation_font_color="#949191")

stacked_fig
#stacked_fig.write_image(image_path / f'assumpt_xpert_compare.png', width=800, height=800,scale=3)


In [13]:
from tb_incubator.plotting import overlay_plots

epi_plots = {}
indicators = ['notification', 'adults_prevalence_pulmonary', 'incidence', 'mortality']

for file in file_suffixes:
    epi_plots[file] = {}  
    for indicator in indicators:
        epi_plots[file][indicator] = plot_output_ranges(
            quantile_outputs[file], targets, 
            [f'{indicator}'], 1, 2010, 2035, 2013, show_legend=False, show_title=False, show_target_data=False
        )
        
notif_none = epi_plots[f'no_extensions_{file_id}']['adults_prevalence_pulmonary']
notif_none_xpert = epi_plots[f'no_cdr_{file_id}']['adults_prevalence_pulmonary']
notif_notif = epi_plots[f'cdr_notif_noxpert_{file_id}']['adults_prevalence_pulmonary']
notif_notif_xpert = epi_plots[f'cdr_notif_{file_id}']['adults_prevalence_pulmonary']
notif_detect = epi_plots[f'cdr_detect_noxpert_{file_id}']['adults_prevalence_pulmonary']
notif_detect_xpert = epi_plots[f'cdr_detect_{file_id}']['adults_prevalence_pulmonary']

fig1 = overlay_plots([notif_none, notif_none_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=True, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig2 = overlay_plots([notif_notif, notif_notif_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=False, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig3 = overlay_plots([notif_detect, notif_detect_xpert], colors=None, title=None, yaxis_title=None, update_layout=False, show_legend=False, 
              plot_names=["Without Xpert improvement", "With Xpert improvement"],
              legend_position="bottom-right")

fig1.add_trace(
    go.Scatter(
        x=targets['adults_prevalence_pulmonary_target'].index,
        y=targets['adults_prevalence_pulmonary_target'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Pulmonary TB prevalence in adults (2013-2014 National Prevalence Survey)",
        legendgroup="Target",
    )
)
fig2.add_trace(
    go.Scatter(
        x=targets['adults_prevalence_pulmonary_target'].index,
        y=targets['adults_prevalence_pulmonary_target'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Pulmonary TB prevalence in adults (2013-2014 National Prevalence Survey)",
        legendgroup="Target",
    )
)
fig3.add_trace(
    go.Scatter(
        x=targets['adults_prevalence_pulmonary_target'].index,
        y=targets['adults_prevalence_pulmonary_target'].values,
        mode="markers",
        marker=dict(size=5, color='black'),
        name="Pulmonary TB prevalence in adults (2013-2014 National Prevalence Survey)",
        legendgroup="Target",
    )
)


#fig.write_image(image_path / f'xpert_compare.png', width=600, height=400,scale=3)

In [14]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

stacked_fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=[f"<b>No CDR</b>", f"<b>CDR on notification</b>", f"<b>CDR on treatment commencement</b>"],  # Update with your actual titles
    vertical_spacing=0.08,  # Adjust spacing between subplots
    shared_xaxes=True  # Share x-axis if they have the same range
)

# Add traces from fig1 to row 1
for trace in fig1.data:
    stacked_fig.add_trace(trace, row=1, col=1)

# Add traces from fig2 to row 2
for trace in fig2.data:
    trace.showlegend = False  # Hide duplicate legends
    stacked_fig.add_trace(trace, row=2, col=1)

# Add traces from fig3 to row 3
for trace in fig3.data:
    trace.showlegend = False  # Hide duplicate legends
    stacked_fig.add_trace(trace, row=3, col=1)

# Update layout
stacked_fig.update_layout(
    height=900,  # Adjust total height
    showlegend=True,
    font_family="Space Grotesk",
    title_font_family="Space Grotesk",
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5
    )
)

# Update y-axis titles if needed
stacked_fig.update_yaxes(title_text="No. of case notification", row=1, col=1)
stacked_fig.update_yaxes(title_text="No. of case notification", row=2, col=1)
stacked_fig.update_yaxes(title_text="No. of case notification", row=3, col=1)

stacked_fig.update_xaxes(title_text="Year", row=3, col=1)  # Only bottom x-axis gets label

stacked_fig.show()


positions = [(1, 1), (2, 1), (3, 1)]
years_data = [
    (2017, "1st inventory study"),
    (2023, "2nd inventory study")
]

for row, col in positions:
    for year, text in years_data:
        stacked_fig.add_vline(x=year, line_dash="dot", row=row, col=col, 
                       line_color="rgba(148, 145, 145, 0.5)", line_width=1,
                       annotation_text=text, annotation_textangle=-90,
                       annotation_position="bottom right",
                       annotation_font_size=9, annotation_font_color="#949191")

stacked_fig
#stacked_fig.write_image(image_path / f'assumpt_xpert_compare_prev.png', width=800, height=800,scale=3)
